# 離反予測を用いた SageMaker Pipelines の ML パイプライン構築

このノートブックは
https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/mlops/sagemaker-pipelines/sagemaker-pipelines-sample/sagemaker-pipelines-sample.ipynb
を簡略化したバージョンです。変更点として
2日目以降の学習の際、前日までのデータを再使用せずその日のデータのみで学習しています。そのためprocessing.pyでのデータのconcat処理と、PipeLineステップでの`PreProcessedXXXXXDataS3UriParam`パラメータを省略しています。

また、変数名が増えて複雑になるのを防ぐためprocess.pyやpostprocess.pyにparameterとして渡していたディレクトリ変数、例えば`PRE_PROCESS_RAW_DATA_INPUT_DIR` (= '/opt/ml/processing/input/raw_data')をやめ、スクリプトおよびジョブ定義時にハードコーディングしています。パイプラインを組む時点で定形処理化が目的であるため、データの構成やインスタンスのディレクトリ構造は変える必要はなく、パラメータ（変数）化する必要はないと判断しました。

ただし、ノートブックでのジョブ定義時のディレクトリ名とスクリプトにハードコーディングしたディレクトリ名が異なるとエラーが出るため、注意してください。
変更しては行けない部分にコメントを記載しました。

※ SageMaker **Studio** Notebook を前提としており、カーネルは Python3(Data Science) をご利用ください。   
※ confirms this notebook works under data science kernel on SageMaker **Studio** notebook

## シナリオ

電話回線の離反データセット（回線ごとのデータと離反した/しなかったの結果のデータセット）を使って、
SageMaker Pipelines を用いたML パイプラインを構築します。
データの詳細については[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)に詳細があります。  

5000 行の 元データを 1666 行ずつ 3 分割し、それぞれ 1 日目に入手するデータ、 2 日目に入手するデータ、 3 日目に入手するデータと仮定し、
* 1 日目は今あるデータを SageMaker Processing, Training をそれぞれ手動で動かす。
* 2 日目は 2 日目のデータで学習し、 1 日目と 2 日目のモデルを比較して、2 日目のほうが精度がよければ 2 日目のモデルを登録(create_model)するのを、パイプラインを構築して実行する。
* 3 日目は 3 日目のデータで学習し、2 日目と 3 日目のモデルを比較して、3 日目のほうが精度がよければ 3 日目のモデルを登録(create_model)するのを、2 日目に作成したパイプラインのパラメータだけを変更して実行する。
* 最後には精度が変わらなかった時のテストとして、同じデータで学習し、パイプラインで精度が変わらなかった場合は新たにモデルが作成(create_model)されていないことをGUIから実行して確認する

In [ ]:
import os, json, sagemaker, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ParameterInteger, ParameterString
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep,TrainingStep
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.transformer import Transformer
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.pipeline import Pipeline

## データ準備   
T/F(離反した、してない) の割合が変わらないように、5000 行のデータを 3 分割する。

In [ ]:
# データをダウンロード
![ -e churn.txt ] && rm churn.txt
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt ./

In [ ]:
# 使用するデータを確認
df = pd.read_csv('./churn.txt')
df.head()

In [ ]:
# データを分割する際、離反データが偏らないように、離反したデータと離反しなかったデータを分けて分割する
df_true = df[df['Churn?']=='True.'].reset_index()
df_false = df[df['Churn?']=='False.'].reset_index()
df_true = df_true.drop(['index'],axis=1)
df_false = df_false.drop(['index'],axis=1)

In [ ]:
# 分割前にシャッフルする
df_true_shuffle = df_true.sample(frac=1, random_state=42)
df_false_shuffle = df_false.sample(frac=1, random_state=42)

In [ ]:
# 3分割する
split_num = 3
split_df_list = []
for i in range(split_num):
    idx_min_true,idx_max_true = i*len(df_true)//split_num,(i+1)*len(df_true)//split_num
    idx_min_false,idx_max_false = i*len(df_false)//split_num,(i+1)*len(df_false)//split_num
    tmp_df = pd.concat([df_true[idx_min_true:idx_max_true],df_false[idx_min_false:idx_max_false]],axis=0)
    split_df_list.append(tmp_df)

In [ ]:
# 分割ファイルをローカルに出力する
RAWDATA_DIR = './raw_data/'
os.makedirs(f'{RAWDATA_DIR}/', exist_ok=True)
local_csvfile_list = []
for i,split_df in enumerate(split_df_list):
    file_name = f'{RAWDATA_DIR}day_{str(i+1)}.csv'
    split_df.to_csv(file_name,index=False)
    local_csvfile_list.append(file_name)
print(*local_csvfile_list)

それぞれのファイルをS3の別フォルダに予めアップロードしておく

In [ ]:
# この PIPELINE_NAME 変数は後ほどパイプラインを作成するときに使うが、前処理の名前などにも流用するため、ここで宣言する
PIPELINE_NAME = 'simple-pipeline'
BUCKET = sagemaker.session.Session().default_bucket()
RAWDATA_SUB_PREFIX = RAWDATA_DIR.replace('./','').replace('/','')

for i, file in enumerate(local_csvfile_list):
    RAWDATA_S3_URI = f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}/day{i+1}'
    rawcsv_s3_uri = sagemaker.s3.S3Uploader.upload(file,RAWDATA_S3_URI)
    print(f'day-{i+1} data was uploaded to {rawcsv_s3_uri}')

del rawcsv_s3_uri

## 1 日目 : 前処理、学習を手作業で行う
### 前処理
前処理は[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)と同じことを SageMaker Processing で行う。コンテナは scikit-learn のビルトインコンテナを利用する

In [ ]:
# Processor 定義
ROLE = get_execution_role()

PRE_PROCESS_JOBNAME = f'{PIPELINE_NAME}-pre-process'
sklearn_processor = SKLearnProcessor(
    base_job_name = PRE_PROCESS_JOBNAME,
    framework_version='0.23-1',
    role=ROLE,
    instance_type='ml.m5.xlarge',instance_count=1
)



In [ ]:
PROCESSED_DATA_SUB_PREFIX = 'preprocessed_data'

sklearn_processor.run(
    code='./preprocess/preprocess.py',
    # ProcessingInput は指定したものを全て S3 から processing インスタンスにコピーされる。 Destination でコピー先を指定できる。
    inputs=[
        ProcessingInput( 
            source=f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}/day1',
            destination='/opt/ml/processing/input' # DO NOT CHANGE
        ),
    ],
    # processing インスタンスの source にあるものを全て S3 に格納する。(processing インスタンス側でこのディレクトリは自動で作成される)
    outputs=[
        ProcessingOutput(
            output_name = 'train',
            source='/opt/ml/processing/output/train', # DO NOT CHANGE
            destination=f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/train'
        ),
        ProcessingOutput(
            output_name = 'valid',
            source='/opt/ml/processing/output/valid', # DO NOT CHANGE
            destination=f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/valid'

        ),
        ProcessingOutput(
            output_name = 'test',
            source='/opt/ml/processing/output/test', # DO NOT CHANGE
            destination=f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/test'

        )
    ],
    # processing インスタンスのどこに csv ファイルが配置されたか、どこにファイルを出力すればよいのか、を
    # コードに渡すための引数
    arguments=[
        '--dummy1',"dummy arguments 1",
        '--dummy2', "dummy arguments 2"
    ]
)

### 学習
xgboost を利用する。ハイパーパラメータは[こちら](https://github.com/aws-samples/amazon-sagemaker-examples-jp/blob/master/xgboost_customer_churn/xgboost_customer_churn.ipynb)と同じにして SageMaker Training で行う。   
コンテナは xgboost のビルトインコンテナを利用する

学習の入力データは前処理ジョブのdestinationで指定したS3パスを明示的に指定します。

In [ ]:
# 前処理結果の S3 URI を取得する
train_csv_s3_uri = f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/train'
valid_csv_s3_uri = f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/valid'
test_csv_s3_uri = f's3://{BUCKET}/{PIPELINE_NAME}-{PROCESSED_DATA_SUB_PREFIX}/day1/test'
print(train_csv_s3_uri)
print(valid_csv_s3_uri)
print(test_csv_s3_uri)

In [ ]:
CONTENT_TYPE='text/csv'
train_s3_input = TrainingInput(train_csv_s3_uri, content_type=CONTENT_TYPE)
valid_s3_input = TrainingInput(valid_csv_s3_uri, content_type=CONTENT_TYPE)

In [ ]:
XGB_CONTAINER_URI = sagemaker.image_uris.retrieve("xgboost", sagemaker.session.Session().boto_region_name, "1.2-1")

In [ ]:
TRAIN_JOBNAME = f'{PIPELINE_NAME}-train'
MODEL_S3_URI = f's3://{BUCKET}/{TRAIN_JOBNAME}'
HYPERPARAMETERS = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.8",
    "objective":"binary:logistic",
    "num_round":"100"
}
xgb = Estimator(
    XGB_CONTAINER_URI,
    ROLE,
    base_job_name = TRAIN_JOBNAME,
    hyperparameters=HYPERPARAMETERS,
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path = MODEL_S3_URI
)

In [ ]:
xgb.fit({'train': train_s3_input, 'validation': valid_s3_input})

後に作成するパイプライン変数のデフォルト値用に、1日めのモデルのs3 uriデータを取得しておきます

In [ ]:
INITIAL_MODEL_S3_URI = xgb.model_data

## 2 日目 : パイプラインを作成する
新しくデータが入ってくるので、追加データも併せてモデルを学習しなおして精度を確認し、精度が上がっていたらモデルを作成しておきます。  
1 日目と同じことをやるので、パイプラインを作成して省力化します。  
2 日目で学習したモデルで精度を比較し、精度が上がっていたらモデル作成、のオペレーションを追加します。

In [ ]:
# helper 関数
# pipeline で利用する名前は camel case を使うのが一般的なので、区切り文字を削除し、頭を大文字にする関数を準備
def to_camel(s_v:str,s_s:str)->str:
    '''
    s_v: camel_case に変えたい文字
    s_s: 区切り文字
    '''
    return ''.join(word.title() for word in s_v.split(s_s))

### Step 定義
パイプラインで実行するには、実行させたい処理を全て [Step](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.Step) で定義する必要があります。  
前処理、学習、後処理、精度評価結果分岐、モデル作成について Step を定義していきます。  
まずは前処理の Step を [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) で定義します。  
また、3 日目以降の実行も見据えて、 3 日目の実行で変わるところは [Parameter](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) を定義しておき、パイプライン実行時に設定できるようにしておきます。  

#### 前処理ステップ
Parameters は文字列なのか数値なのかで使用するものが変わります。  
今回は文字列で定義する箇所が 3 日以降実行時に変更されるので、[ParameterString](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.parameters.ParameterString) を使用しますが、整数なら [ParameterInteger](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.parameters.ParameterInteger), 浮動小数ならば [ParameterFloat](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.parameters.ParameterFloat) を利用します。  
それぞれデフォルト値が必須ですので、 1 日目の前処理結果の S3 URI を利用します。

In [ ]:
# 前処理ステップ定義

sklearn_processor = SKLearnProcessor(
    base_job_name = PRE_PROCESS_JOBNAME,
    framework_version='0.23-1',
    role=ROLE,
    instance_type='ml.m5.xlarge',instance_count=1
)



# Pipeline 実行時に渡すパラメータ設定
rawcsv_s3_uri_param = ParameterString(name='RawCsvS3UriParam',default_value=f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}/day1',)

# 前処理を実行する ProcessingStep 定義
pre_process_step = ProcessingStep(
    code='./preprocess/preprocess.py',
    name=f'{to_camel(PRE_PROCESS_JOBNAME,"-")}Step', #'MyPipelinePreProcessStep'

    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=rawcsv_s3_uri_param, # Parameter を渡します。翌日以降はまた URI が変わるため。
            destination='/opt/ml/processing/input' # DO NOT CHANGE
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name = 'train',
            source='/opt/ml/processing/output/train', # DO NOT CHANGE
        ),
        ProcessingOutput(
            output_name = 'valid',
            source='/opt/ml/processing/output/valid', # DO NOT CHANGE
        ),
        ProcessingOutput(
            output_name = 'test',
            source='/opt/ml/processing/output/test', # DO NOT CHANGE
        )
    ],
    job_arguments=[
        '--dummy1',"dummy arguments 1",
        '--dummy2', "dummy arguments 2"
    ]
)

#### 学習用ステップ
学習用の Step である [TraningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) を定義します。  
学習の入力には、先程の前処理の結果を使いますが、先程 `ProcessingStep` で定義した `pre_process_step` インスタンスにあるプロパティから、前段の処理の出力の URI を連携できるので、それを利用します。

そのため前段のProcessingStepではProcessingOutputにdestinationを指定していません。その場合ProcessingOutputのS3上の出力は
`s3://<default-bucket-name>/<job-name>/output/<output-name>` に出力されます

In [ ]:
# 学習ステップ定義
xgb = Estimator(
    XGB_CONTAINER_URI,
    ROLE,
    base_job_name = TRAIN_JOBNAME,
    hyperparameters=HYPERPARAMETERS,
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path = MODEL_S3_URI
)

train_step = TrainingStep(
    name=f'{to_camel(TRAIN_JOBNAME,"-")}Step',
    estimator=xgb,
    inputs={
        "train": TrainingInput(
            s3_data=pre_process_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, # 前の処理の output_name が 'train' の出力 S3 URI を利用
            content_type=CONTENT_TYPE #'text/csv'
        ),
        "validation": TrainingInput(
            s3_data=pre_process_step.properties.ProcessingOutputConfig.Outputs["valid"].S3Output.S3Uri, # 前の処理の output_name が 'valid' の出力 S3 URI を利用
            content_type=CONTENT_TYPE
        )
    },
)

#### 後処理用ステップ
後処理で前回学習したモデルと今回学習したモデルの性能評価を AUC で行います。
評価用スクリプトは予め  `./postprocessing/postprocess.py` に格納してあります。  
後処理用のステップにも ProcessingStep を利用します。2 つのモデルを評価する後処理ステップを定義しますが、  
評価ステップの入力は、1. 前回の学習で出来上がったモデルと、2. 今回の学習で出来上がったモデルの S3 URI と、3. 評価用データです。  
それぞれ下記の通りに入力を定義します。
1. 前回学習したモデルは毎回変わるため、 `ParameterString` で Parameter として設定しておきます。デフォルト値が必須ですが、 1 日目に学習したモデルの URI を `INITIAL_MODEL_S3_URI = xgb.model` で格納していたので、それを利用します。  
1. 今回の学習で出来上がったモデルの S3 URI は 学習ステップ時に前処理ステップの結果を取得したのと同様に、前段の `train_step` プロパティから取得できるのでそれを利用します。
1. 評価用データは前処理の結果の `test.csv` を利用するので、前処理ステップの `pre_process_step` のプロパティから取得します。

ProcessingInputは今回データ1つ、モデル2つの合計3つを使うため、それぞれinput以下に別のディレクトリを指定しておきます


`destination='/opt/ml/processing/input/data'` 
`destination='/opt/ml/processing/input/thistime_train_model'`
`destination='/opt/ml/processing/input/previoustime_train_model'`

ProcessingOutputは評価結果の1種類になります


In [ ]:
lasttime_train_model_s3_uri_param = ParameterString(name='LasttimeTrainModelS3UriParam',default_value=INITIAL_MODEL_S3_URI)

POST_PROCESS_JOBNAME = f'{PIPELINE_NAME}-post-process'

EVALUATION_FILE = 'evaluation.json'
eval_processor = ScriptProcessor(
    base_job_name = f'{POST_PROCESS_JOBNAME}',
    image_uri=XGB_CONTAINER_URI,
    command=['python3'],
    instance_type='ml.m5.xlarge',
    instance_count=1,
    role=ROLE,
)
eval_report = PropertyFile(
    name='EvaluationReport',
    output_name='Evaluation',
    path=EVALUATION_FILE
)

eval_step = ProcessingStep(
    code='./postprocess/postprocess.py',
    name=f'{to_camel(POST_PROCESS_JOBNAME,"-")}EvalStep',
    processor=eval_processor,
    inputs=[
        ProcessingInput(
            source=pre_process_step.properties.ProcessingOutputConfig.Outputs['test'].S3Output.S3Uri,
            destination='/opt/ml/processing/input/data' # DO NOT CHANGE
        ),
        ProcessingInput(
            source=train_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/input/thistime_train_model' # DO NOT CHANGE
        ),
        ProcessingInput(
            source=lasttime_train_model_s3_uri_param,
            destination='/opt/ml/processing/input/lasttime_train_model' # DO NOT CHANGE
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='Evaluation',
            source='/opt/ml/processing/output' # DO NOT CHANGE
        ),
    ],
    property_files=[eval_report],
    job_arguments=[
        '--output-file',EVALUATION_FILE
    ]
)

#### モデル作成ステップ
評価後、前回の学習した結果より AUC がよかったら、今回の学習結果を SageMaker にそのモデルを作成します。  
前回より AUC がよかったら、の処理についてはこの後記述します。分岐処理もステップで表現しますが、分岐ステップに分岐後の step を登録する必要があるためです。  
[CreateModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.CreateModelStep) で定義します。入力はもちろん学習ステップのプロパティから取得したモデルの S3 URI です。


In [ ]:
model = Model(
    image_uri=XGB_CONTAINER_URI,
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker.session.Session(),
    role=ROLE,
)
model_inputs = CreateModelInput(
    instance_type="ml.m5.large",
)
create_model_step = CreateModelStep(
    name=f'{to_camel(PIPELINE_NAME,"-")}CreateModelStep',
    model=model,
    inputs=model_inputs,
)

#### 分岐ステップ
前回の学習より今回の学習のほうが AUC がよかった場合の分岐ステップを記述します。  
[後処理スクリプト](./postprocess/postprocess.py)で、 `evaluation.json` を出力するよう記述してあり、 jsonの中にそれぞれのモデルの AUC と、今回の学習のほうがよかった場合は 1 、悪かった場合は 0 を残す、 `classification_metrics.model_change` キーを残しているので、  
その値が 1 だった場合は、`True` を返す `cond_e` を [ConditionEquals](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionEquals)で定義します。

作成した `cond_e` を [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.condition_step.ConditionStep) の `if_steps` に登録します。  
`if_steps` はリストを受け取りますが、 and 条件なので気をつけてください。  
また、条件が False だった場合の挙動を記述したい場合は `else_step` を記述してください。

In [ ]:
cond_e = ConditionEquals(
    left=JsonGet(
        step=eval_step,
        property_file=eval_report,
        json_path="classification_metrics.model_change",
    ),
    right=1,
)

cond_step = ConditionStep(
    name=f'{to_camel(PIPELINE_NAME,"-")}ConditionStep',
    conditions=[cond_e],
    if_steps=[create_model_step],
    else_steps=[], 
)

### パイプライン生成
最後に今まで作成した parameters と step をつなげてパイプラインを [Pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.condition_step.ConditionStep) で生成します。  
pipeline インスタンスの `start ` メソッドでパイプラインを実行できます。parameter のデフォルト値で実行します。

In [ ]:
pipeline = Pipeline(
    name=to_camel(PIPELINE_NAME,"-"),
    parameters=[
        rawcsv_s3_uri_param,
        lasttime_train_model_s3_uri_param
    ],
    steps=[
        pre_process_step,
        train_step,
        eval_step,
        cond_step,
    ],
)

In [ ]:
# パイプライン定義を確認
print(json.loads(pipeline.definition()))

In [ ]:
# パイプラインを SageMaker Pipelines に登録
pipeline.upsert(role_arn=ROLE)

パイプラインを実行するパラメータを指定します。2日めは生データに2日めのデータを指定します。

In [ ]:
input_param = {
    'RawCsvS3UriParam': f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}/day2', # 2日目のraw data
}

print(json.dumps(input_param,indent=4))

In [ ]:
# パイプライン実行。非同期処理
execution = pipeline.start(parameters = input_param)

In [ ]:
# 実行しているパイプラインの中身を表示
execution.describe()

In [ ]:
# パイプラインが完了するまで待つ
execution.wait()

## 3 日目 : 作成済のパイプラインにパラメータを与えて動かす
2 日目に作成したパイプライン作成工数の投資回収をします。すでに構築済のパイプラインを呼び出して、 3 日目用のパラメータを与えて実行します。  

### 構築済パイプラインの呼び出し
構築済のパイプラインを呼び出すのは [sagemaker.workflow.pipeline.Pipeline](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline.Pipeline)を利用し、作成済のパイプライン名を `name`  引数に指定します。  
パイプライン構築時と同じクラスですが、 `steps` や `parameters` 引数を指定せずに、名前だけ構築済のものを指定すれば、構築済のパイプラインを呼び出せます

In [ ]:
built_pipeline = Pipeline(
    name=to_camel(PIPELINE_NAME,"-"), # 存在しているパイプライン名
)
print(built_pipeline.describe())

### 入力パラメータの確認
引き渡すパラメータを確認します。describe メソッドの結果から Parameters キーを参照します。

In [ ]:
for param in json.loads(built_pipeline.describe()['PipelineDefinition'])['Parameters']:
    print(param)

### 入力パラメータの設定
* `RawCsvS3Param` : 先程アップロードした 3 日目の csv の S3 URI を指定します。
* `LasttimeTrainModelS3UriParam` : 先程学習したモデルの URI を boto3 を利用して取得します。

In [ ]:
# 先程のパイプライン実行でできたモデルの S3 URI を取得する
import boto3
client = boto3.client('sagemaker')
lasttime_train_model_s3_uri=''
for step in execution.list_steps():
    if step['StepName'] == f'{to_camel(TRAIN_JOBNAME,"-")}Step':
        last_train_arn = step['Metadata']['TrainingJob']['Arn']
lasttime_train_model_s3_uri = client.describe_training_job(TrainingJobName=last_train_arn.split('/')[-1])['ModelArtifacts']['S3ModelArtifacts']
print(lasttime_train_model_s3_uri)

パラメータを dict 形式に格納します。

In [ ]:
input_param = {
    'RawCsvS3UriParam': f's3://{BUCKET}/{PIPELINE_NAME}-{RAWDATA_SUB_PREFIX}/day3', # 3日目のraw data
    'LasttimeTrainModelS3UriParam': lasttime_train_model_s3_uri, # 前回のexecution (Pipelineジョブ)で作成されたモデルのs3 uri
}

print(json.dumps(input_param,indent=4))

### パイプライン実行
あとは paramters 引数に input_param を指定して start するとパイプラインが実行できます。  
一度パイプラインを作成してしまえば、一連のプロセスをすぐに動かすことができます。

In [ ]:
# パイプラインをキック
execution = built_pipeline.start(parameters = input_param, execution_display_name="execution-day3")
# パイプラインの完了を待つ
#execution.wait()

## 最後に if 分岐の False ルート確認と GUI から実行
3 日目と全く同じデータを学習し、3 日と同じモデルで評価します。同じ精度が出ればモデル作成ステップが動かないので、それを利用して分岐ステップが機能しているかを確認しまうｓ。
パイプラインは GUI でも実行可能なので、 GUI で実行してみます。3 日目のパイプライン実行と入力が一箇所だけ、 `LasttimeTrainModelS3UriParam` だけ変更する必要があるので、3 日目の学習のモデル S3 URI を取得します。

In [ ]:
# 先程のパイプライン実行でできたモデルの S3 URI を取得する
import boto3
client = boto3.client('sagemaker')
lasttime_train_model_s3_uri=''
for step in execution.list_steps():
    if step['StepName'] == f'{to_camel(TRAIN_JOBNAME,"-")}Step':
        last_train_arn = step['Metadata']['TrainingJob']['Arn']
lasttime_train_model_s3_uri = client.describe_training_job(TrainingJobName=last_train_arn.split('/')[-1])['ModelArtifacts']['S3ModelArtifacts']
print(lasttime_train_model_s3_uri)

先程の `input_json` を書き換えます。

In [ ]:
input_param['LasttimeTrainModelS3UriParam']=lasttime_train_model_s3_uri

下記の出力をテキストエディタなどにコピーしておきます。この中身を GUI に入力します。

In [ ]:
print(json.dumps(input_param,indent=4))

1. 左のペインから三角形の頂点に○がついているアイコンを選択し、表示されるプルダウンで `Pipelines` を選択します。  
  ![](media/1.png)  

2. 表示されるパイプライン一覧から作成したパイプライン(このノートブックで特にいじっていなければ `MyPipeline`) をダブルクリックします。  
  ![](media/2.png)  

3. パイプラインの実行履歴が表示されます。右上にある `Start an Execution` をクリックして、パイプラインの実行設定を行います。  
  ![](media/3.png)  

4. `Name` のテキストボックに任意の実行名を入力し、他のテキストボックスには先程コピーした内容をそれぞれ入力して、 `Start` をクリックします。  
  ![](media/4.png)  

5. `Executing` と表示されたパイプラインの実行が作成されます（即座に反映されないことがあるので、表示されない場合は待ってください）。`Execiting` と表示された実行をダブルクリックします。
  ![](media/5.png)  

6. 実行の様子がわかります。各ステップを選択すると、ログや artifact を確認できます。
  ![](media/6.png)  
  
7. 実行が完了すると Status が緑に変わります。また、 3 日目と同じモデルが出来上がり、同じ精度が出ているので、`ConditionStep` の結果が `False` になり、 `CreateModelStep` が実行されていないことが確認できます。（今までの実行を確認すると、緑になっています）
  ![](media/7.png)  



## あとかたづけ
パイプラインの削除は パイプラインインスタンスにある delete メソッドを利用して削除します。  
使用したり作成したデータとモデルは S3 に保管されているのでマネジメントコンソールや boto3 などで別途削除してください。

In [ ]:
built_pipeline.delete()